<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#潜在语义模型练习" data-toc-modified-id="潜在语义模型练习-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>潜在语义模型练习</a></span></li><li><span><a href="#潜在语义分析（LSA）" data-toc-modified-id="潜在语义分析（LSA）-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>潜在语义分析（LSA）</a></span><ul class="toc-item"><li><span><a href="#奇异值分解（SVD）" data-toc-modified-id="奇异值分解（SVD）-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>奇异值分解（SVD）</a></span></li><li><span><a href="#非负矩阵分解" data-toc-modified-id="非负矩阵分解-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>非负矩阵分解</a></span></li></ul></li><li><span><a href="#概率潜在语义分析" data-toc-modified-id="概率潜在语义分析-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>概率潜在语义分析</a></span></li><li><span><a href="#潜在狄利克雷分配" data-toc-modified-id="潜在狄利克雷分配-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>潜在狄利克雷分配</a></span><ul class="toc-item"><li><span><a href="#Gibbs抽样" data-toc-modified-id="Gibbs抽样-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Gibbs抽样</a></span></li></ul></li></ul></div>

# 潜在语义模型练习

主要参考赵航的书

# 潜在语义分析（LSA）

我们的文档包含两个主题：一个是粒子物理，常用词汇为"meson", "photon"。
另外一个是生活，常用词汇为"girl", "boy"。

In [1]:
import numpy as np

# documents set
documents=[
    "girl boy",
    "boy girl",
    "girl boy",
    "a boy girl",
    "the boy girl",
    "girl",
    "boy",
    
    "the meson photon",
    "a meson photon",
    "photon",
    "meson",
    "photon meson",
]

# setup terms set
terms_set = set()
for doc in documents:
    terms = doc.split()
    for term in terms:
        terms_set.add(term)
        
print("%d terms found in %d documents."%(len(terms_set), len(documents)))
terms_vec=[]
terms_vec.extend(terms_set)
print("terms:", terms_vec)


# setup terms index
terms_index = { }
for idx, term in enumerate(terms_set):
    terms_index[term] = idx


6 terms found in 12 documents.
terms: ['the', 'photon', 'a', 'girl', 'boy', 'meson']


我们使用频率逆文档(TFIDF)来表示term-document矩阵。

In [2]:
# terms-documents matrix
tf_ij=np.zeros((len(terms_set),len(documents)))
tf_j=np.zeros(len(documents))  # number of terms in document j 
df_i=np.zeros(len(terms_set))  # number of documents which contain term i
df=len(documents)


term_doc_mat = np.zeros([len(terms_set), len(documents)])
for j, doc in enumerate(documents):
    terms = doc.split()
    tf_j[j] = len(terms)
    
    add_to_df_i=np.zeros(len(terms_set))
    for term in terms:
        i = terms_index[term]
        tf_ij[i][j] = tf_ij[i][j] + 1
        add_to_df_i[i] = 1
        
    df_i = df_i + add_to_df_i


# X in hang's book
term_doc_mat = tf_ij/tf_j*np.log(df/df_i).reshape(df_i.shape[0], 1)
        
with np.printoptions(precision=3):
    print("term-document (%dX%d) matrix setup"%(term_doc_mat.shape[0],term_doc_mat.shape[1]))
    print(term_doc_mat)

term-document (6X12) matrix setup
[[0.    0.    0.    0.    0.597 0.    0.    0.597 0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.366 0.366 1.099 0.    0.549]
 [0.    0.    0.    0.597 0.    0.    0.    0.    0.597 0.    0.    0.   ]
 [0.347 0.347 0.347 0.231 0.231 0.693 0.    0.    0.    0.    0.    0.   ]
 [0.347 0.347 0.347 0.231 0.231 0.    0.693 0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.366 0.366 0.    1.099 0.549]]


## 奇异值分解（SVD）

将term-documents矩阵分解为 u s vT
保留其中主值最大的部分。
奇异值分解的正交性要求，矩阵的部分数值为0，如何解释呢？

In [3]:
from numpy.linalg import svd

u,s,vT = svd(term_doc_mat)

#print("term-topic matrix:\n", u)
#print("topic-document matrix:\n", vT)
#print("main value:\n", s)

def print_mat(term_topic_mat, topic_document_mat):
    print("doc-topic matrix")
    print(" %20s   %10s  %10s"%("","topic0", "topic1"))
    for i, doc in enumerate(documents):
        print("`%20s'  %+10f  %+10f"%(doc, topic_document_mat[0][i], topic_document_mat[1][i]))

    print("term-topic matrix")
    print(" %20s   %10s  %10s"%("","topic0", "topic1"))

    for i, term in enumerate(terms_vec):
        print(" %20s   %+10f  %+10f"%(term, term_topic_mat[i][0], term_topic_mat[i][1]))

print_mat(u, vT)

doc-topic matrix
                            topic0      topic1
`            girl boy'   -0.020776   +0.383461
`            boy girl'   -0.020776   +0.383461
`            girl boy'   -0.020776   +0.383461
`          a boy girl'   -0.081727   +0.346752
`        the boy girl'   -0.081727   +0.346752
`                girl'   -0.020776   +0.383461
`                 boy'   -0.020776   +0.383461
`    the meson photon'   -0.386577   +0.034020
`      a meson photon'   -0.386577   +0.034020
`              photon'   -0.478051   -0.085636
`               meson'   -0.478051   -0.085636
`        photon meson'   -0.478051   -0.085636
term-topic matrix
                            topic0      topic1
                  the    -0.178288   +0.186259
               photon    -0.682644   -0.095174
                    a    -0.178288   +0.186259
                 girl    -0.047023   +0.675462
                  boy    -0.047023   +0.675462
                meson    -0.682644   -0.095174


## 非负矩阵分解

将term-documents矩阵$X$分解为 $WH$，其中$W$和$H$为（任意元素）非负矩阵。
目标为优化
$$\sum_{ij}(WH-X)_{ij}^2$$
优化方法采用“乘法更新规则”。

In [4]:
from numpy.linalg import norm

ntopics = 2
print("We want %d topics"%ntopics)

W = np.random.rand(len(terms_vec), ntopics)
H = np.random.rand(ntopics, len(documents))
# W * H -> X (term-document matrix)
for i in range(100):
    H1 = H * ((W.transpose().dot(term_doc_mat)) / (W.transpose().dot(W).dot(H)))
    dH = H1 - H
    H = H1
    W1 = W*((term_doc_mat.dot(H.transpose())) / (W.dot(H.dot(H.transpose()))))
    dW = W1 - W
    W = W1
    #print(np.power(dH,2).sum())
    #print(np.power(dW,2).sum())
    #print(np.power(term_doc_mat-W.dot(H),2).sum())

print_mat(W, H)

We want 2 topics
doc-topic matrix
                            topic0      topic1
`            girl boy'   +0.000000   +0.040593
`            boy girl'   +0.000000   +0.040593
`            girl boy'   +0.000000   +0.040593
`          a boy girl'   +0.023796   +0.079060
`        the boy girl'   +0.023796   +0.079060
`                girl'   +0.000000   +0.040593
`                 boy'   +0.000000   +0.040593
`    the meson photon'   +0.171537   +0.257857
`      a meson photon'   +0.171537   +0.257857
`              photon'   +0.454545   +0.000000
`               meson'   +0.000000   +0.636973
`        photon meson'   +0.220744   +0.308828
term-topic matrix
                            topic0      topic1
                  the    +0.244073   +0.247510
               photon    +2.366617   +0.000086
                    a    +0.244073   +0.247510
                 girl    +0.000000   +0.163212
                  boy    +0.000000   +0.163212
                meson    +0.000673   +1.615948


# 概率潜在语义分析

概率潜在语义分析即潜在语义分析概率化版本，是一种生成模型。

设文档集合为$D=\{d_j\}, j < |D|$，单词集合为$W=\{w_i\}, i < |W|$，文档$d_j$也是单词的序列，序列长度为$|d_j|$（重复单词多次计数。）

概率模型给出文档$d_j$中某一单词位置（比如第一个单词），单词$w_i$出现的概率$P(w_i, d_j)$为
$$
P(w_i, d_j) = 
\sum_k P(w_i, z_k) P(z_k, d_j)
$$
其中$P(z_k, d_j)$为文档$d_j$中某一单词选取话题$z_k$的概率，$P(w_i, z_k)$为话题$z_k$中单词$w_i$出现的概率。

设文档$d_j$第$i^\prime$个单词是单词$w_{i(d_j, i^\prime)}$（$i^\prime < |d_j|$），
则文档$d_j$生成的概率为
$$
\prod_{i^\prime < |d_j|} P(w_{i(d_j, i^\prime)}, d_j)
=\prod_{i < |W|} { P(w_i, d_j)^{n(w_i, d_j)} }
$$
$n(w_i, d_j)$为单词$i$在文档$j$中出现的频次。
所有文档的生成概率的对数为
$$
L = \sum_j \sum_{i} n(w_i, d_j) \log P(w_i, d_j)
$$
优化方法采用EM算法。


In [5]:
ntopics = 2
print("We want %d topics"%ntopics)


Pwz = np.random.rand(len(terms_vec), ntopics)
Pzd = np.random.rand(ntopics, len(documents))

nwd = np.zeros((len(terms_vec), len(documents))) # number of word `w` in document `d`

# setup terms set
for j, doc in enumerate(documents):
    terms = doc.split()
    for term in terms:
        i = terms_index[term]
        nwd[i][j] = nwd[i][j] + 1


for i in range(100):
    
    
    
    nd = nwd.sum(axis=0)
    
    # add 1E-6 to avoid 0
    # we need to inverse the elements of this matrix
    Pwd = Pwz.dot(Pzd) + np.full((len(terms_vec),len(documents)), 1E-6)
    
    Pwz_new = (nwd/Pwd).dot(Pzd.transpose()) * Pwz
    Pwz_new = Pwz_new / Pwz_new.sum(axis=0)
    
    
    Pzd_new = (nwd/Pwd).transpose().dot(Pwz).transpose() * Pzd
    Pzd_new = Pzd_new / nd
        
    #print(np.power(Pwz_new-Pwz,2).sum())
    #print(np.power(Pzd_new-Pzd,2).sum())
    #print(np.power(Pzd_new,2).sum())
    
    Pwz = Pwz_new
    Pzd = Pzd_new


print_mat(Pwz, Pzd)

We want 2 topics
doc-topic matrix
                            topic0      topic1
`            girl boy'   +0.999998   +0.000000
`            boy girl'   +0.999998   +0.000000
`            girl boy'   +0.999998   +0.000000
`          a boy girl'   +0.666665   +0.333327
`        the boy girl'   +0.666910   +0.333082
`                girl'   +0.999998   +0.000000
`                 boy'   +0.999998   +0.000000
`    the meson photon'   +0.000000   +0.999996
`      a meson photon'   +0.000000   +0.999996
`              photon'   +0.000000   +0.999997
`               meson'   +0.000000   +0.999997
`        photon meson'   +0.000000   +0.999997
term-topic matrix
                            topic0      topic1
                  the    +0.000061   +0.166615
               photon    +0.000000   +0.333355
                    a    +0.000000   +0.166676
                 girl    +0.499969   +0.000000
                  boy    +0.499969   +0.000000
                meson    +0.000000   +0.333355


# 潜在狄利克雷分配

设文档集合为$D=\{d_m\}, m<|D|$，单词集合为$W=\{w_i\},i < |W|$，文档$d_j$也是单词的序列，序列长度为$|d_m|$（重复单词多次计数。）。
话题向量为$\mathbf{z}=\{z_m\},m < |D|$。
单词向量为$\mathbf{w} = \{w_{mn}\}, m < |D|, n< |d_m|$。
话题向量为$\mathbf{z} = \{z_{mn}\}, m < |D|, n< |d_m|$。
话题向量先验计数为$\mathbf{\alpha} = \{\alpha_{mk}\}, m < |D|, k < K$。
话题词向量先验计数为$\mathbf{\beta} = \{\beta_{ki}\}, k < K, i < |W|$。


$$
P(\mathbf{w}, \mathbf{z}, \mathbf{\theta},\mathbf{\phi}| \alpha, \beta) = 
\left[\prod_k \text{Dir} (\phi_k|\beta)\right]
\left[\prod_m \text{Dir}(\theta_m|\alpha)\right]
\left[
\prod_{mn} \text{Cat}(z_{mn}|\theta_m) 
\text{Cat}(w_{mn}|\phi_{z_{mn}})
\right]
$$

我们想要求 $P(\mathbf{z},\theta,\phi|\mathbf{w})$
$$P(\mathbf{z},\theta,\phi|\mathbf{w})
=
P(\theta,\phi|\mathbf{z},\mathbf{w})
P(\mathbf{z},\mathbf{w})
/P(\mathbf{w})
=
P(\theta,\phi|\mathbf{z},\mathbf{w})
P(\mathbf{z}|\mathbf{w})
$$
其中

$$
P(\mathbf{z},\mathbf{w}) = \prod_{k} \frac{B(n_k+\beta)}{B(\beta)} \prod_{m} \frac{B(n_m+\alpha)}{B(\alpha)}
$$

和

$$
P(\theta,\phi|\mathbf{z},\mathbf{w})
= \left[\prod_m \text{Dir}(\theta_m|n_m+\alpha)\right] \left[\prod_k \text{Dir}(\phi_k|n_k+\beta)\right]
$$




## Gibbs抽样

$P(\mathbf{z}|\mathbf{w})$可以通过Gibbs抽样方法得到。
注意根据狄利克雷分布，
$\theta_m$ 的期望为 $$(n_m + \alpha)/\sum_k (n_{mk}+\alpha_{mk})$$
$\phi_k$ 的期望为 $$(n_k + \beta)/\sum_{i} (n_{ki}+\beta_{ki})$$

In [6]:
ntopics = 2
print("We want %d topics"%ntopics)


alpha=0.2
beta=0.2

n_mk=np.zeros((len(documents), ntopics))
n_kv=np.zeros((ntopics, len(terms_vec)))
n_m=np.zeros(len(documents))
n_k=np.zeros(ntopics)
beta=1
alpha=1

lengths=np.zeros(len(documents),dtype=np.int)
for m, doc in enumerate(documents):
    lengths[m] = len(doc.split())    
k_mn=np.zeros((len(documents), np.max(lengths)),dtype=np.int)
    
iters=10
theta_mk_iters=np.zeros((iters, len(documents), ntopics))
phi_kv_iters=np.zeros((iters, ntopics, len(terms_vec)))
    

def print_matrix():
    print("doc-topic counting")
    print(n_mk)
    print("terms counting in each doc")
    print(n_m)
    print("document length")
    print(lengths)
    print("topic-words counting")
    print(n_kv)
    print("terms counting in each topic")
    print(n_k)
    

# intialization randomly
# assign each word a topic
for m, doc in enumerate(documents):
    terms = doc.split()
    for n, term in enumerate(terms):
        v = terms_index[term]
        k = int(np.random.randint(ntopics))        
        n_mk[m, k] += 1 
        n_m[m] += 1 
        n_kv[k,v] += 1 
        n_k[k] += 1
        k_mn[m, n] = k
        
        
# generate P(z|w) sample
def Gibbs_step():

    for m, doc in enumerate(documents):
        terms = doc.split()
        for n, term in enumerate(terms):
            v = terms_index[term]

            # current topic
            k = k_mn[m, n]            
            n_mk[m, k] -= 1
            #n_m[m] -= 1
            n_kv[k, v] -= 1
            n_k[k] -= 1
            
            n_kv_ = (n_kv[:,v] + beta) / (n_kv[k,:] + beta).sum()
            n_mk_ = (n_mk[m,:] + alpha) / (n_mk[m,:] + alpha).sum()
            
            #print(n_kv_)
            #print(n_mk_)
            
            #print(n_kv)
            acc = (n_kv_*n_mk_).cumsum()/(n_kv_*n_mk_).sum()
            p = np.random.rand()
            #print(acc)
            kp = np.searchsorted(acc, p, side="left")
            #print(n_kv_*n_mk_,acc, p, kp)
                        
            k_mn[m, n] = kp
            n_mk[m, kp] += 1
            #n_m[m] += 1
            n_kv[kp, v] += 1
            n_k[kp] += 1

            
# warnming
for _ in range(1000):
    Gibbs_step()

# average
for it in range(iters):
    Gibbs_step()
    
    theta_mk = (n_mk + alpha)/(n_mk + alpha).sum(axis=1, keepdims=True)
    phi_kv = (n_kv + beta)/(n_kv + beta).sum(axis=1, keepdims=True)
    #print_mat(phi_kv.transpose(), theta_mk.transpose())

    theta_mk_iters[it,:,:] = theta_mk
    phi_kv_iters[it,:,:] = phi_kv
    


print_mat(phi_kv_iters.mean(axis=0).transpose(), theta_mk_iters.mean(axis=0).transpose())

We want 2 topics
doc-topic matrix
                            topic0      topic1
`            girl boy'   +0.325000   +0.675000
`            boy girl'   +0.275000   +0.725000
`            girl boy'   +0.275000   +0.725000
`          a boy girl'   +0.220000   +0.780000
`        the boy girl'   +0.280000   +0.720000
`                girl'   +0.366667   +0.633333
`                 boy'   +0.366667   +0.633333
`    the meson photon'   +0.400000   +0.600000
`      a meson photon'   +0.440000   +0.560000
`              photon'   +0.533333   +0.466667
`               meson'   +0.566667   +0.433333
`        photon meson'   +0.575000   +0.425000
term-topic matrix
                            topic0      topic1
                  the    +0.112203   +0.113434
               photon    +0.257702   +0.112893
                    a    +0.135446   +0.099467
                 girl    +0.125711   +0.272875
                  boy    +0.137431   +0.276298
                meson    +0.231506   +0.125033
